In [1]:
import os
import csv
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
from data_cleaning_utils import clean_csv, filter_columns
from utils import set_pandas_display_options
set_pandas_display_options()

# Cleaning dataframes
At this section we will clean all DF's using following statements.
- Function `clean_csv` used before read csv into DF.
    - Clean quotes.
    - Clean spaces.
    - Delete semicolon and replace each one by colon.

- Clean each DF columns (Coluns in Notion, only yes take as necessary col's).
- Deleting duplicates from caracteristiques.


In [2]:
current_path = os.getcwd()
data_path = current_path + '/data'

In [3]:
# Setting up input and output files
files = {
    "caracteristiques":["carcteristiques-{year}.csv", "carcteristiques-{year}-cleaned.csv"],
    "lieux":["lieux-{year}.csv", "lieux-{year}-cleaned.csv"],
    "usagers":["usagers-{year}.csv", "usagers-{year}-cleaned.csv"],
    "vehicules":["vehicules-{year}.csv", "vehicules-{year}-cleaned.csv"]
}

# Run cleaning
year = '2022'
for g_file in files.values():
    input_file = f'{data_path}/2022/{g_file[0].replace("{year}",year)}'  
    output_file = f'{data_path}/cleaned/{year}/{g_file[1].replace("{year}", year)}'    
    clean_csv(input_file=input_file,
              output_file=output_file)     

In [9]:
caracteristiques = pd.read_csv(filepath_or_buffer=f'{data_path}/cleaned/{year}/{files["caracteristiques"][1].replace("{year}", year)}')
lieux = pd.read_csv(filepath_or_buffer=f'{data_path}/cleaned/{year}/{files["lieux"][1].replace("{year}", year)}')
usagers = pd.read_csv(filepath_or_buffer=f'{data_path}/cleaned/{year}/{files["usagers"][1].replace("{year}", year)}')
vehicules = pd.read_csv(filepath_or_buffer=f'{data_path}/cleaned/{year}/{files["vehicules"][1].replace("{year}", year)}')

/var/folders/ln/nzb_k4696z1b57xs0pgft7740000gn/T/ipykernel_78364/1945647949.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  lieux = pd.read_csv(filepath_or_buffer=f'{data_path}/cleaned/{year}/{files["lieux"][1].replace("{year}", year)}')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94493 entries, 0 to 94492
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   num_acc      94493 non-null  int64  
 1   id_vehicule  94493 non-null  int64  
 2   num_veh      94493 non-null  object 
 3   senc         94493 non-null  int64  
 4   catv         94493 non-null  int64  
 5   obs          94493 non-null  int64  
 6   obsm         94493 non-null  int64  
 7   choc         94493 non-null  int64  
 8   manv         94493 non-null  int64  
 9   motor        94493 non-null  int64  
 10  occutc       817 non-null    float64
dtypes: float64(1), int64(9), object(1)
memory usage: 7.9+ MB


In [5]:
lieux_col_drop = ['voie', 'v1', 'v2', 'circ', 'nbv', 'pr', 'pr1', 'vosp', 'prof', 'plan', 'lartpc', 'larrout', 'situ']
carac_col_drop = ['adr']
usag_col_drop = ['secu1', 'secu2', 'secu3']
vehic_col_drop = ['senc', 'motor', 'occutc']

caracteristiques = filter_columns(caracteristiques, carac_col_drop)
lieux = filter_columns(lieux, lieux_col_drop)
usagers = filter_columns(usagers, usag_col_drop)
vehicules = filter_columns(vehicules, vehic_col_drop)


KeyError: "['num_Veh'] not found in axis"

In [6]:
# Checking and deleting duplicates from caracteristiques
caract_dubs = caracteristiques.duplicated(subset=['num_acc'])
# last check identify if there are duplicates in the data using boolean values
print('joined data total duplicated: %s / %s' % (len(caracteristiques[caract_dubs]), len(caracteristiques)))
# drop duplicates
caracteristiques = caracteristiques.drop_duplicates(subset=['num_acc'])
print('Droped duplicates: %s / %s' % (len(caracteristiques[caract_dubs]), len(caracteristiques)))

joined data total duplicated: 0 / 55302
Droped duplicates: 0 / 55302


## NOTE
- Identified: no unecessary duplicates,
- Identified, all data is normalized (no categorical values).
- Identified: There's no treshold variance

# Merge DF's

Each Df will be merged in left join:

- merge_1 = Caracteristiques with lieux using `num_acc`.
- merge_2 = Usagers with vehicules using `num_acc`, `id_vehicule` and `num_veh`.

Each result will be merged to from result merge_1 to merge_2 using `num_acc`



In [51]:
merge_1 = caracteristiques.merge(lieux, on="num_acc", how='left')
merge_2 = usagers.merge(vehicules, on=['num_acc', 'id_vehicule', 'num_veh'], how='left')
full_data = merge_2.merge(merge_1, on='num_acc', how='left')
# full_data.info()


In [52]:
from numpy import nan 
# Final cleaning

# Filled all NaN values with nan
full_data = full_data.fillna(nan)

# Replacing all an_nais (born year) values by integer values and filling NaN values with 0
full_data['an_nais'] = full_data['an_nais'].fillna(0).astype(int)

full_data['date'] = pd.to_datetime(full_data['jour'].astype(str) + '-' + full_data['mois'].astype(str) + '-' + full_data['an'].astype(str) + ' ' + full_data['hrmn'].astype(str), format='%d-%m-%Y %H:%M')
full_data['timestamp'] = full_data['date'].astype(int) // 10**9
data_cols_drop = ['jour', 'mois', 'an', 'hrmn', 'date', 'num_veh', 'adr', 'voie', 'v1', 'v2', 'occutc', 'lartpc']
full_data = filter_columns(full_data, data_cols_drop)

# Replacing row specific errors
full_data.replace({'A': '10', 'B': '11', "(1)" : '1'}, regex=False, inplace=True)
full_data.replace({'2A' : '96', '2B' : '97'}, regex=True, inplace = True) #On remplace les corses
full_data.drop(full_data[full_data['com'] == "N/C"].index, inplace=True)
full_data.drop(full_data[full_data['nbv'] == '#ERREUR'].index, inplace=True)

# Removing NaN indexes
full_data.dropna(inplace=True)

full_data.head()

,num_acc,id_usager,id_vehicule,place,catu,grav,sexe,an_nais,trajet,secu1,secu2,secu3,locp,actp,etatp,senc,catv,obs,obsm,choc,manv,motor,lum,dep,com,agg,int,atm,col,lat,long,catr,circ,nbv,vosp,prof,pr,pr1,plan,larrout,surf,infra,situ,vma,timestamp
0,202200000001,1099700,813952,1,1,3,1,2008,5,2,8,-1,-1,-1,-1,1,2,0,2,1,9,1,1,26,26198,2,3,1,3,44.559420,4.725720,4,2,2,0,1,1,1,1,-1.0,1,0,1,50,1666196100
1,202200000001,1099701,813953,1,1,1,1,1948,5,1,8,-1,-1,-1,-1,1,7,0,2,2,1,1,1,26,26198,2,3,1,3,44.559420,4.725720,4,2,2,0,1,1,1,1,-1.0,1,0,1,50,1666196100
2,202200000002,1099698,813950,1,1,4,1,1988,9,1,0,-1,0,0,-1,2,7,0,2,8,15,1,1,25,25204,2,3,1,3,46.925810,6.346200,4,2,2,0,1,1,1,1,-1.0,1,0,1,50,1666254840
3,202200000002,1099699,813951,1,1,1,1,1970,4,1,0,-1,0,0,-1,2,10,0,2,1,1,1,1,25,25204,2,3,1,3,46.925810,6.346200,4,2,2,0,1,1,1,1,-1.0,1,0,1,50,1666254840
4,202200000003,1099696,813948,1,1,1,1,2002,0,1,0,-1,-1,-1,-1,2,7,0,2,1,2,1,1,22,22360,2,6,1,2,48.493162,-2.760439,3,-1,2,0,1,0,0,1,-1.0,1,5,1,50,1666286100


In [53]:
# Values dep, com, num_veh, actp are non integer values and is not possible to convert them to integer values
full_data.info()


<class 'pandas.core.frame.DataFrame'>
Index: 126658 entries, 0 to 126661
Data columns (total 45 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   num_acc      126658 non-null  int64  
 1   id_usager    126658 non-null  int64  
 2   id_vehicule  126658 non-null  int64  
 3   place        126658 non-null  int64  
 4   catu         126658 non-null  int64  
 5   grav         126658 non-null  int64  
 6   sexe         126658 non-null  int64  
 7   an_nais      126658 non-null  int64  
 8   trajet       126658 non-null  int64  
 9   secu1        126658 non-null  int64  
 10  secu2        126658 non-null  int64  
 11  secu3        126658 non-null  int64  
 12  locp         126658 non-null  int64  
 13  actp         126658 non-null  object 
 14  etatp        126658 non-null  int64  
 15  senc         126658 non-null  int64  
 16  catv         126658 non-null  int64  
 17  obs          126658 non-null  int64  
 18  obsm         126658 non-null 

In [54]:
# saving data
full_data.to_csv(f'{data_path}/full_data.csv', index=False)